In [18]:
from sqlalchemy import create_engine, text
import os
import sys
import setting

SQL_SETTING = setting.SQL_SETTING

engine = create_engine(f"mysql+pymysql://{SQL_SETTING['user']}:{SQL_SETTING['password']}@{SQL_SETTING['host']}:3306/{SQL_SETTING['database']}?charset={SQL_SETTING['charset']}")

conn = engine.connect()

In [19]:

import os
import pandas as pd

# csv들 불러오기
target_dir = "/data3/"

# 실행시키는 위치
base_dir = os.getcwd()

divider = "/"
# linux, mac
if base_dir[-1] != "/":
  divider = "\\"
  target_dir = target_dir.replace("/", "\\")

dir = base_dir + divider + target_dir + divider

users = pd.read_csv(dir + "user.csv")
posts = pd.read_csv(dir + "post.csv")
interests = pd.read_csv(dir + "interest.csv")
interactions = pd.read_csv(dir + "interaction.csv")
categories = pd.read_csv(dir + "category.csv")


In [20]:
print(users)

      id       nickname gender  age
0      0     셋둘꿀벌여왕0000      M   31
1      1     셋둘책책공주0001      F   35
2      2     둘둘또가스왕0002      M   26
3      3    하나둘참새왕자0003      M   15
4      4      지존책책왕0004      F   33
..   ...            ...    ...  ...
995  995    지존꽹가리여왕0995      F   48
996  996    셋둘잠만보여왕0996      M   25
997  997    최강야도란친구0997      F   38
998  998    둘둘꼬부기공주0998      F   25
999  999  하나둘고라파덕왕자0999      M   16

[1000 rows x 4 columns]


In [21]:
BATCH_SIZE = 10000

def insert_db(df, table_name):
  # 인덱스 0부터 시작할 수 있게
  sql = 'set SQL_MODE = "NO_AUTO_VALUE_ON_ZERO"'
  conn.execute(text(sql))
  print(f"{table_name} 테이블 생성 중...")

  for iteration in range (0, len(df), BATCH_SIZE):
    # 테이블 칼럼 세팅
    sql = f"INSERT INTO {table_name} ("

    for i in range(len(df.columns)):
      if i == len(df.columns) - 1:
        sql += f"{df.columns[i]}"
      else:
        sql += f"{df.columns[i]}, "

    sql += ") \nVALUES\n"

    # 값 넣기
    for i in range(iteration, min(iteration + BATCH_SIZE, len(df))):
      sql += f"("
      for j in range(len(df.columns)):
        if j == len(df.columns) - 1:
          sql += f"'{df.iloc[i, j]}'"
        else:
          sql += f"'{df.iloc[i, j]}', "
      sql += "),"

    sql = sql[:-1] + ";"

    conn.execute(text(sql))
    conn.commit()
    done = min(iteration + BATCH_SIZE, len(df) - 1)
    sys.stdout.write(f"\r{done} / {len(df)} ({done / len(df) * 100:.2f}%)")
    sys.stdout.flush()

  sys.stdout.write(f"\r{len(df)} / {len(df)} ({100:.2f}%)")
  sys.stdout.flush()
  print()
  print()

def delete_db(table_name):
  print(f"{table_name} 테이블 삭제 중...")

  sql = f"DELETE FROM {table_name};"
  conn.execute(text(sql))

  # AUTO_INCREMENT 초기화
  sql = f'alter table {table_name} AUTO_INCREMENT=0;'
  conn.execute(text(sql))

posts_refined = pd.merge(left = posts, right = categories, how="left", left_on="category", right_on="tag")
posts_refined = posts_refined.drop(columns=["category"])

posts_refined = posts_refined[['id_x', 'writer_id', 'title', 'content', 'created_at', 'like_count', 'unlike_count', 'comment_count', 'id_y']]
posts_refined.rename(columns={'id_x':'id', 'id_y':'category_id'}, inplace=True)

likes = interactions[interactions["type"].isin(["L", "U"])].copy()
likes.reset_index(drop=True, inplace=True)
likes["positive"] = (likes["type"] == "L").astype(int)  # "L"이면 1, 아니면 0
likes["updated_at"] = likes["created_at"]  
likes = likes.drop(columns=["type"])
likes["id"] = likes.index
likes = likes[["id", "created_at", "positive", "post_id", "user_id", "updated_at"]]

# 인덱스 값을 id 칼럼으로 넣음
interactions["id"] = interactions.index
# type이 L 이랑 U 인건 제외
interactions = interactions[~interactions["type"].isin(["L", "U"])].copy()
interests["id"] = interests.index

delete_db("curating_users")
insert_db(users, "curating_users")

delete_db("curating_posts")
insert_db(posts_refined, "curating_posts")

delete_db("curating_interests")
insert_db(interests, "curating_interests")

delete_db("curating_interactions")
insert_db(interactions, "curating_interactions")

delete_db("curating_categories")
insert_db(categories, "curating_categories")

delete_db("curating_likes")
insert_db(likes, "curating_likes")

print("Complete!")

curating_users 테이블 삭제 중...
curating_users 테이블 생성 중...
1000 / 1000 (100.00%)

curating_posts 테이블 삭제 중...
curating_posts 테이블 생성 중...
1716 / 1716 (100.00%)

curating_interests 테이블 삭제 중...
curating_interests 테이블 생성 중...
2137 / 2137 (100.00%)

curating_interactions 테이블 삭제 중...
curating_interactions 테이블 생성 중...
222178 / 222178 (100.00%)

curating_categories 테이블 삭제 중...
curating_categories 테이블 생성 중...
15 / 15 (100.00%)

curating_likes 테이블 삭제 중...
curating_likes 테이블 생성 중...
50296 / 50296 (100.00%)

Complete!
